In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 10

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-14 13:13:37 [__init__.py:239] Automatically detected platform cpu.


2025-04-14 13:13:37,973	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg, True)

2025-04-14 13:13:39.672 | INFO     | trl_wrapper.trainer_wrapper:__init__:297 - Setting padding side to: right


In [3]:
wrapper.init_model()

2025-04-14 13:13:39.740 | INFO     | trl_wrapper.trainer_wrapper:init_model:312 - Using device: mps
2025-04-14 13:13:39.740 | INFO     | trl_wrapper.trainer_wrapper:init_model:326 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [5]:
wrapper.init_data_module()

2025-04-14 13:33:54.857 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:352 - Using chat template override: smollmv2
Map: 100%|██████████| 1184/1184 [00:00<00:00, 24167.72 examples/s]


In [6]:
wrapper.data_module.train_dataset

Dataset({
    features: ['prompt'],
    num_rows: 10647
})

In [7]:
wrapper.init_trainer()

2025-04-14 13:34:00.076 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:390 - Saving output to: ./runs/04-14-13-34-993080-smollm2-135m-instruct-
2025-04-14 13:34:00.077 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:413 - Initializing trainer, run_name: 04-14-13-34-993080-smollm2-135m-instruct-, wandb project: writing-grpo
2025-04-14 13:34:00.078 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:496 - Using vllm: False


In [ ]:
wrapper.train()

2025-04-14 14:49:48.337 | INFO     | trl_wrapper.trainer_wrapper:train:659 - Starting training.
2025-04-14 14:50:11.679 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 2 completions with gemini-2.0-flash


kwargs: {}


2025-04-14 14:50:14.743 | INFO     | model.reasoning:logger_reward:229 - Generations:
2025-04-14 14:50:14.743 | INFO     | model.reasoning:logger_reward:231 - ========================================
2025-04-14 14:50:14.743 | INFO     | model.reasoning:logger_reward:232 - Prompt 0: --------------------
2025-04-14 14:50:14.744 | INFO     | model.reasoning:logger_reward:233 - Write a paragraph in a style reminiscent of suspenseful fantasy fiction, focusing on a group of characters exploring a bizarre and unstable architectural space. The tone should be a mix of bewilderment, unease, and growing dread.

The paragraph should depict a space where architectural styles and eras are colliding, creating illogical and dangerous layouts. Include details of rooms and corridors merging in impossible ways, with items and furnishings from different time periods fused together. Focus on the characters' growing sense of disorientation and the realization that the space is becoming increasingly unstable

scores: [{'Adherence to Instructions': 10.0, 'Believable Character Actions': 8.0, 'Nuanced Characters': 7.0, 'Consistent Voice/Tone of Writing': 12.0, 'Imagery and Descriptive Quality': 11.0, 'Elegant Prose': 10.0, 'Emotionally Engaging': 9.0, 'Emotionally Complex': 5.0, 'Coherent': 14.0, 'Meandering': 8.0, 'Weak Dialogue': 1.0, "Tell-Don't-Show": 10.0, 'Unsurprising or Uncreative': 8.0, 'Amateurish': 7.0, 'Purple Prose': 6.0, 'Overwrought': 5.0, 'Incongruent Ending Positivity': 1.0, 'Unearned Transformations': 1.0, 'Well-earned Lightness or Darkness': 1.0, 'Sentences Flow Naturally': 14.0, 'Overall Reader Engagement': 8.0, 'Overall Impression': 9.0}, {'Adherence to Instructions': 5.0, 'Believable Character Actions': 1.0, 'Nuanced Characters': 1.0, 'Consistent Voice/Tone of Writing': 10.0, 'Imagery and Descriptive Quality': 8.0, 'Elegant Prose': 6.0, 'Emotionally Engaging': 3.0, 'Emotionally Complex': 1.0, 'Coherent': 10.0, 'Meandering': 12.0, "Tell-Don't-Show": 15.0, 'Unsurprising or 

2025-04-14 14:50:40.881 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 2 completions with gemini-2.0-flash


...

kwargs: {}


2025-04-14 16:25:24.788 | INFO     | model.reasoning:logger_reward:229 - Generations:
2025-04-14 16:25:24.789 | INFO     | model.reasoning:logger_reward:231 - ========================================
2025-04-14 16:25:24.789 | INFO     | model.reasoning:logger_reward:232 - Prompt 0: --------------------
2025-04-14 16:25:24.789 | INFO     | model.reasoning:logger_reward:233 - Write a paragraph in a gritty, noir style, focusing on a protagonist named Moss and his tense encounter with a character named Oliver. The tone should be cynical and world-weary, with hints of moral ambiguity. The scene is set in a baker's kitchen, contrasting warmth with the protagonist's cold predicament.

Moss, injured and disheveled, confronts Oliver after being betrayed. He is clearly in a weakened state, reflecting a reversal of fortune. Oliver should be portrayed as smug and superior, yet also subtly anxious. Imply Oliver obtained information through harsh methods, questioning the morality of his actions.

In

scores: [{'Adherence to Instructions': 2.0, 'Believable Character Actions': 2.0, 'Nuanced Characters': 2.0, 'Consistent Voice/Tone of Writing': 5.0, 'Imagery and Descriptive Quality': 3.0, 'Elegant Prose': 3.0, 'Emotionally Engaging': 1.0, 'Emotionally Complex': 1.0, 'Coherent': 4.0, 'Meandering': 15.0, 'Weak Dialogue': 20.0, "Tell-Don't-Show": 18.0, 'Unsurprising or Uncreative': 19.0, 'Amateurish': 18.0, 'Purple Prose': 8.0, 'Overwrought': 10.0, 'Incongruent Ending Positivity': 0.0, 'Unearned Transformations': 0.0, 'Well-earned Lightness or Darkness': 2.0, 'Sentences Flow Naturally': 7.0, 'Overall Reader Engagement': 1.0, 'Overall Impression': 1.0}, {'Adherence to Instructions': 1.0, 'Believable Character Actions': 1.0, 'Nuanced Characters': 1.0, 'Consistent Voice/Tone of Writing': 2.0, 'Imagery and Descriptive Quality': 2.0, 'Elegant Prose': 2.0, 'Emotionally Engaging': 1.0, 'Emotionally Complex': 1.0, 'Coherent': 2.0, 'Meandering': 18.0, 'Weak Dialogue': 20.0, "Tell-Don't-Show": 18.

2025-04-14 16:25:49.277 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 2 completions with gemini-2.0-flash


kwargs: {}


In [10]:
wrapper.data_module.reward_functions()[0](["hi"], ["hi"])

2025-04-14 14:45:32.005 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 1 completions with gemini-2.0-flash


kwargs: {}
scores: [{'Adherence to Instructions': 20.0, 'Believable Character Actions': 0.0, 'Nuanced Characters': 0.0, 'Consistent Voice/Tone of Writing': 20.0, 'Imagery and Descriptive Quality': 0.0, 'Elegant Prose': 0.0, 'Emotionally Engaging': 0.0, 'Emotionally Complex': 0.0, 'Coherent': 20.0, 'Meandering': 20.0, 'Weak Dialogue': 0.0, "Tell-Don't-Show": 0.0, 'Unsurprising or Uncreative': 1.0, 'Amateurish': 2.0, 'Purple Prose': 20.0, 'Overwrought': 20.0, 'Incongruent Ending Positivity': 20.0, 'Unearned Transformations': 20.0, 'Well-earned Lightness or Darkness': 0.0, 'Sentences Flow Naturally': 20.0, 'Overall Reader Engagement': 1.0, 'Overall Impression': 10.0}]
score_sums: [194.0]


[194.0]